# Chat Models

## 1_chat_model_basic

In [ ]:
%pip install -q python-dotenv langchain langchain-community langchain-ollama

In [ ]:
from dotenv import load_dotenv

In [ ]:
# Load environment variables from .env
load_dotenv()

In [ ]:
from langchain_ollama.llms import OllamaLLM

In [ ]:
# Create a OllamaLLM model
model = OllamaLLM(base_url="http://localhost:11434", model="llama3.2", temperature=0)

In [ ]:
# Invoke the model with a message
result = model.invoke("What is 81 divided by 9?")

In [ ]:
print("Full result:")
print(result)

## 2_chat_model_basic_conversation

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [ ]:
from langchain_ollama import ChatOllama

In [ ]:
model = ChatOllama(
        base_url="http://localhost:11434",
        model="llama3.2",
        temperature=0)

In [ ]:
# SystemMessage:
#   Message for priming AI behavior, usually passed in as the first of a sequenc of input messages.
# HumanMessagse:
#   Message from a human to the AI model.
messages = [
    SystemMessage(content="Solve the following math problems"),
    HumanMessage(content="What is 81 divided by 9?"),
]

In [ ]:
# Invoke the model with messages
result = model.invoke(messages)

In [ ]:
print(f"Answer from AI: {result}")

In [ ]:
print(f"Answer from AI: {result.content}")

In [ ]:
# AIMessage:
#   Message from an AI.
messages = [
    SystemMessage(content="Solve the following math problems"),
    HumanMessage(content="What is 81 divided by 9?"),
    AIMessage(content="81 divided by 9 is 9."),
    HumanMessage(content="What is 10 times 5?"),
]

In [ ]:
# Invoke the model with messages
result = model.invoke(messages)
print(f"Answer from AI: {result.content}")

In [ ]:
# Invoke the model with messages
result = model.invoke("Summarize our conversation so far")
print(f"Answer from AI: {result.content}")

In [ ]:
# AIMessage:
#   Message from an Human.
messages = [
    SystemMessage(content="Solve the following math problems"),
    HumanMessage(content="What is 81 divided by 9?"),
    AIMessage(content="81 divided by 9 is 9."),
    HumanMessage(content="What is 10 times 5?"),
    AIMessage(content="10 times 5 is 50."),
    HumanMessage(content="Summarize our conversation so far")
]

In [ ]:
# Invoke the model with messages
result = model.invoke(messages)
print(f"Answer from AI: {result.content}")

### 3_chat_model_alternatives

In [ ]:
%pip install -q langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [ ]:
openai_model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
messages = [
    SystemMessage(content="Solve the following math problems"),
    HumanMessage(content="What is 81 divided by 9?"),
]

In [ ]:
# Invoke the model with messages
result = openai_model.invoke(messages)
print(f"Answer from OpenAI: {result.content}")

## 4_chat_model_conversation_with_user

In [ ]:
# Use a list to store messages
chat_history = []

In [ ]:
# Set an initial system message (optional)
system_message = SystemMessage(content="You are a helpful AI assistant.")
chat_history.append(system_message)  # Add system message to chat history

In [ ]:
# Chat loop
while True:
    query = input("You: ")
    if query.lower() == "exit":
        break
    chat_history.append(HumanMessage(content=query))  # Add user message

    # Get AI response using history
    result = model.invoke(chat_history)
    response = result.content
    chat_history.append(AIMessage(content=response))  # Add AI message

    print(f"AI: {response}")


print("---- Message History ----")
print(chat_history)

## 5_chat_model_conversation_memory

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
# store is a dictionary that maps session IDs to their corresponding chat histories.
store = {}  # memory is maintained outside the chain

In [ ]:
# A function that returns the chat history for a given session ID.
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [ ]:
#  Define a RunnableConfig object, with a `configurable` key. session_id determines thread
config = {"configurable": {"session_id": "1"}}

In [ ]:
conversation = RunnableWithMessageHistory(
    model,
    get_session_history,
)

In [ ]:
result = conversation.invoke(
    "Hi I'm Srijan Dubey.",  # input or query
    config=config,
)
print(f"Answer from OpenAI: {result.content}")

In [ ]:
result = conversation.invoke(
    "What is my name?",  # input or query
    config=config,
)
print(f"Answer from OpenAI: {result.content}")

In [ ]:
store

In [ ]:
%pip install -q langchain-community SQLAlchemy

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [ ]:
# Function to get session history
# create sync sql message history by connection_string
def get_session_history(session_id: str):
    return SQLChatMessageHistory(
        session_id=session_id, 
        connection="sqlite:///memory.db"
    )

In [ ]:
config = {"configurable": {"session_id": "session1"}}

In [ ]:
conversation_sql = RunnableWithMessageHistory(
    model,
    get_session_history,
)

In [ ]:
result = conversation_sql.invoke(
    "Hi I'm Srijan Dubey.",  # input or query
    config=config,
)
print(f"Answer from OpenAI: {result.content}")

In [ ]:
result = conversation_sql.invoke(
    "What is my name?",  # input or query
    config=config,
)
print(f"Answer from OpenAI: {result.content}")

In [ ]:
config = {"configurable": {"session_id": "session2"}}

In [ ]:
result = conversation_sql.invoke(
    "What is my name?",  # input or query
    config=config,
)
print(f"Answer from OpenAI: {result.content}")

In [ ]:
# create sync sql message history by connection_string
message_history = get_session_history("session1")

In [ ]:
message_history.get_messages()